# Last known position

This notebook retrieves the last know position of each satallite_id by making a simple query to the DB.

In [1]:
from datetime import datetime
import json

from influxdb_client import InfluxDBClient

In [2]:
url = "http://influxdb:8086"
token = "my-admin-token"
org = "my-org"
bucket = "my-bucket"

client = InfluxDBClient(url=url, token=token, org=org)

Now we load the json data and extract all the satelites id

In [3]:
starklink_datafile = 'starlink_historical_data.json'

with open(starklink_datafile, 'r') as data_file:
    json_data = data_file.read()

starklink_data = json.loads(json_data)

satellite_ids = {data['id'] for data in starklink_data}

Now we can easily select group and sort by time.

In [4]:
for satellite_id in satellite_ids:
    
    query_api = client.query_api()
    df_result = query_api.query_data_frame(
        f"""
        from(bucket: "my-bucket")
          |> range(start: -3y)
          |> filter(fn: (r) => r["_measurement"] == "sat_pos_{satellite_id}")
          |> last()
        """)
    
    display(df_result.head())
    break
    

,result,table,_start,_stop,_time,_value,_field,_measurement
0,_result,0,2019-01-20 09:35:24.374668+00:00,2022-01-20 03:35:24.374668+00:00,2021-01-26 13:26:11+00:00,-23.5382,lat,sat_pos_5eed7715096e590006985799
1,_result,1,2019-01-20 09:35:24.374668+00:00,2022-01-20 03:35:24.374668+00:00,2021-01-26 13:26:11+00:00,75.0000,lon,sat_pos_5eed7715096e590006985799


# TODO: Create a module to simplify/wrap the flux query setups

In [5]:
import json
from influxdb import InfluxDBClient

class SatelliteEngineException(Exception):
    pass

class SatelliteEngine():
    """Some nice doc string"""
    def __init__(self, influxdb_dns, port, user, password, db_name, satellites_ids):
        self.client = InfluxDBClient(
        'influxdb',
        8086,
        'admin',
        'password',
        'db1')
        
        self.client.switch_database(db_name)
        self.satellites_ids = satellites_ids

        
    
    def upsert(self, payload):
        
        for load in payload:
            keys = load.keys()
            if not 'measurement' in keys:
                raise SatelliteEngineException('aa')
                
        return self.client.write_points(payload)
        
        
    def retrieve_last(self, n=1):
        retrieved_data = []
        for satellite_id in self.satellites_ids:
            query = (f'SELECT satellite_id, latitude, longitude FROM sat_pos_{satellite_id}  ORDER BY time LIMIT {n}')
            response = client.query(query)
            retrieved_data.append(response.raw['series'][0]['values'][0])
            
        return retrieved_data
            


In [ ]:
satellites_ids = [
    '5eed7714096e59000698566b',
    '5eed7716096e5900069857dd',
    '5eed7714096e590006985684',
    '5eed7715096e590006985767',
    '5fa3109229f50d0006226d97',
    '5eed7716096e5900069857e8',
    '5eed7714096e590006985694',
    '5eed7716096e5900069857e4',
    '5f36cb59bd888300062740cb']

satellite_engine = SatelliteEngine(
    influxdb_dns= 'influxdb',
    port=8086,
    user='admin',
    password='password',
    db_name='starklink_db',
    satellites_ids=satellites_ids)

satellite_engine.retrieve_last()

returned_longitudes = [row[2] for row in satellite_engine.retrieve_last() ]

In [ ]:
assert( len(returned_longitudes) == 9) 